In [56]:
import os
import sys
import csv
import random
import itertools
from operator import itemgetter
from collections import defaultdict
import numpy as np
import scipy
import scipy.spatial.distance
from numpy.linalg import svd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import utils

In [57]:
numDimen = 300
gloveFile = 'glove.6B.' + str(numDimen) + 'd.txt'
GLOVE = utils.glove2dict(os.path.join('glove.6B', gloveFile))

In [58]:
def randvec(n=50, lower=-0.5, upper=0.5):
    """Returns a random vector of length `n`"""
    return np.array([random.uniform(lower, upper) for i in range(n)])

def getGloveVec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE.get(w, randvec(n=numDimen))

def sentenceToGlove(words):
    result = [0] * numDimen
    for word in words:
        result = np.add(result, getGloveVec(word))
    return result

def getCosineSim(u, v):        
    """Cosine distance between 1d np.arrays `u` and `v`, which must have 
    the same dimensionality. Returns a float."""
    return scipy.spatial.distance.cosine(u, v)

In [59]:
sourceStrs = {}
goldTargetStrs = {}
targetStrs = []
devIndexes = []
vocab = [0]

f = open('data/movie_50000.txt')
for line in f:
    vocab.append(line.rstrip('\n'))
f.close()

f = open('data/dev.src_string.txt')
for i, line in enumerate(f):
    sourceStrs[i+1] = line.strip().split()
f.close()

f = open('data/dev.tgt_string.txt')
for i, line in enumerate(f):
    goldTargetStrs[i+1] = line.strip().split()
f.close()

f = open('data/index_dev.txt')
for line in f:
    devIndexes.append(int(line.rstrip('\n')))
f.close()

f = open('data/rearank_target_dev.txt')
for line in f:
    words = []
    vocabIndexes = line.strip().split()
    for index in vocabIndexes:
        words.append(vocab[int(index)])
    targetStrs.append(words)
f.close()

In [60]:
srcGloveDistances = []
for i, target in enumerate(targetStrs):
    source = sourceStrs[devIndexes[i]]
    cosineSim = getCosineSim(sentenceToGlove(source), sentenceToGlove(target))
    srcGloveDistances.append(cosineSim)
                
tgtGloveDistances = []
for i, target in enumerate(targetStrs):
    goldTarget = goldTargetStrs[devIndexes[i]]
    cosineSim = getCosineSim(sentenceToGlove(goldTarget), sentenceToGlove(target))
    tgtGloveDistances.append(cosineSim)

In [63]:
# np.savetxt("dev_src_glove_dist.txt", srcGloveDistances, fmt='%f')
# np.savetxt("dev_tgt_glove_dist.txt", tgtGloveDistances, fmt='%f')